In [ ]:
!pip install librosa soundfile torchaudio datasets transformers accelerate evaluate tensorboardX jiwer

In [10]:
from datasets import Dataset, Audio, Value
import torch
import evaluate
from dataclasses import dataclass
from typing import Any, Dict, List, Union
from datasets import DatasetDict, Audio, load_from_disk, concatenate_datasets
from transformers.models.whisper.english_normalizer import BasicTextNormalizer
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [39]:
import sagemaker.huggingface
import sagemaker

sess = sagemaker.Session()
sagemaker_session_bucket = None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"Role arn: {role}")
print(f"Bucket: {sess.default_bucket()}")
print(f"Region: {sess.boto_region_name}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
Role arn: arn:aws:iam::589513478258:role/service-role/AmazonSageMaker-ExecutionRole-20231108T202830
Bucket: sagemaker-eu-north-1-589513478258
Region: eu-north-1


In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch

# Specify your AWS SageMaker role
role = get_execution_role()

training_data_s3 = 's3://indian-accent-english/svarah.tar' 

sagemaker_session = sagemaker.Session()

estimator = PyTorch(entry_point='train.py',  
                    role=role,
                    instance_count=1,         
                    instance_type='t2.micro', 
                    py_version='py3',
                    framework_version='1.8.0',
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'epochs': 10,
                        'batch-size': 64,
                    })

estimator.fit({'training': training_data_s3})